### Silver Layer — Clean & Standardized Data (SQL Only)


#### Environment Setup

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Medallion-SQL").getOrCreate()
print("Spark version:", spark.version)
DATA_BASE = "/content"


#### Create `silver` database

In [ ]:
spark.sql("""CREATE DATABASE IF NOT EXISTS silver""")

#### Customers: trim, normalize labels, deduplicate by latest `cst_create_date`

In [ ]:
spark.sql("""DROP TABLE IF EXISTS silver.crm_cust_info""")

In [ ]:
spark.sql("""CREATE TABLE silver.crm_cust_info
USING parquet
AS
WITH stg AS (
  SELECT
    CAST(cst_id AS STRING)      AS cst_id,
    TRIM(cst_firstname)         AS cst_firstname,
    TRIM(cst_lastname)          AS cst_lastname,
    UPPER(TRIM(cst_gndr))       AS cst_gndr_raw,
    UPPER(TRIM(cst_marital_status)) AS cst_marital_status_raw,
    CAST(cst_create_date AS TIMESTAMP) AS cst_create_date
  FROM bronze.crm_cust_info
),
norm AS (
  SELECT
    cst_id, cst_firstname, cst_lastname,
    CASE WHEN cst_gndr_raw IN ('F','FEMALE') THEN 'Female'
         WHEN cst_gndr_raw IN ('M','MALE')   THEN 'Male'
         ELSE 'n/a' END AS cst_gndr,
    CASE WHEN cst_marital_status_raw IN ('S','SINGLE') THEN 'Single'
         WHEN cst_marital_status_raw IN ('M','MARRIED') THEN 'Married'
         ELSE 'n/a' END AS cst_marital_status,
    cst_create_date
  FROM stg
),
latest AS (
  SELECT * FROM (
    SELECT n.*,
           ROW_NUMBER() OVER (PARTITION BY n.cst_id ORDER BY n.cst_create_date DESC NULLS LAST) AS rn
    FROM norm n
  ) t WHERE rn = 1
)
SELECT cst_id, cst_firstname, cst_lastname, cst_gndr, cst_marital_status, cst_create_date
FROM latest
WHERE cst_id IS NOT NULL""")

Preview `silver.crm_cust_info`

In [ ]:
spark.sql("""SELECT * FROM silver.crm_cust_info LIMIT 20""").show(20, truncate=False)

Row count `silver.crm_cust_info`

In [ ]:
spark.sql("""SELECT COUNT(*) AS rows FROM silver.crm_cust_info""").show(truncate=False)

#### Products: trim/cast to canonical types (uses actual columns from `prd_info.csv`)

In [ ]:
spark.sql("""DROP TABLE IF EXISTS silver.crm_prd_info""")

In [ ]:
spark.sql("""CREATE TABLE silver.crm_prd_info
USING parquet
AS
SELECT
  CAST(prd_id AS STRING)        AS prd_id,
  CAST(prd_key AS STRING)       AS prd_key,
  TRIM(prd_nm)                  AS prd_nm,
  CAST(prd_cost AS DOUBLE)      AS prd_cost,
  TRIM(prd_line)                AS prd_line,
  CAST(prd_start_dt AS TIMESTAMP) AS prd_start_dt,
  CAST(prd_end_dt   AS TIMESTAMP) AS prd_end_dt
FROM bronze.crm_prd_info""")

Preview `silver.crm_prd_info`

In [ ]:
spark.sql("""SELECT * FROM silver.crm_prd_info LIMIT 20""").show(20, truncate=False)

Row count `silver.crm_prd_info`

In [ ]:
spark.sql("""SELECT COUNT(*) AS rows FROM silver.crm_prd_info""").show(truncate=False)

#### Sales: cast numeric/date fields and rename to canonical names

In [ ]:
spark.sql("""DROP TABLE IF EXISTS silver.crm_sales_details""")

In [ ]:
spark.sql("""CREATE TABLE silver.crm_sales_details
USING parquet
AS
SELECT
  CAST(sls_ord_num AS STRING)          AS sale_id,
  CAST(sls_cust_id AS STRING)          AS cst_id,
  CAST(sls_prd_key AS STRING)          AS prd_key,
  CAST(sls_quantity AS INT)            AS qty,
  CAST(sls_price AS DOUBLE)            AS price,
  CAST(sls_sales AS DOUBLE)            AS amount,
  CAST(sls_order_dt AS TIMESTAMP)      AS order_dt,
  CAST(sls_ship_dt AS TIMESTAMP)       AS ship_dt,
  CAST(sls_due_dt AS TIMESTAMP)        AS due_dt
FROM bronze.crm_sales_details
WHERE sls_cust_id IS NOT NULL AND sls_prd_key IS NOT NULL""")

Preview `silver.crm_sales_details`

In [ ]:
spark.sql("""SELECT * FROM silver.crm_sales_details LIMIT 20""").show(20, truncate=False)

Row count `silver.crm_sales_details`

In [ ]:
spark.sql("""SELECT COUNT(*) AS rows FROM silver.crm_sales_details""").show(truncate=False)

#### Passthrough `erp_cust_az12` to Silver

In [ ]:
spark.sql("""DROP TABLE IF EXISTS silver.erp_cust_az12""")

In [ ]:
spark.sql("""CREATE TABLE silver.erp_cust_az12 USING parquet AS SELECT * FROM bronze.erp_cust_az12""")

In [ ]:
spark.sql("""SELECT COUNT(*) AS rows FROM silver.erp_cust_az12""").show(truncate=False)

#### Passthrough `erp_loc_a101` to Silver

In [ ]:
spark.sql("""DROP TABLE IF EXISTS silver.erp_loc_a101""")

In [ ]:
spark.sql("""CREATE TABLE silver.erp_loc_a101 USING parquet AS SELECT * FROM bronze.erp_loc_a101""")

In [ ]:
spark.sql("""SELECT COUNT(*) AS rows FROM silver.erp_loc_a101""").show(truncate=False)

#### Passthrough `erp_px_cat_g1v2` to Silver

In [ ]:
spark.sql("""DROP TABLE IF EXISTS silver.erp_px_cat_g1v2""")

In [ ]:
spark.sql("""CREATE TABLE silver.erp_px_cat_g1v2 USING parquet AS SELECT * FROM bronze.erp_px_cat_g1v2""")

In [ ]:
spark.sql("""SELECT COUNT(*) AS rows FROM silver.erp_px_cat_g1v2""").show(truncate=False)